## My First Advent of Code !!

### Day 1

In [1]:
# WARNING : this can take a while to run, depending on repeat length

def day_one(repeat):

    count=0
    count_list=[]
    i=0

    print("start")

    for _ in range(repeat):
        with open("./input.txt",'r') as filein:
            for line in filein:
                if line[0] == "-":
                    count= count - float(line[1:])
                    if count in count_list:
                        print ("repeat found for "+str(count))
                        return
                    else:
                        count_list.append(count)

                elif line[0] == "+":
                    count = count + float(line[1:])
                    if count in count_list:
                        print ("repeat found for "+str(count))  
                        return
                    else:
                        count_list.append(count)

                else:
                    print ("urgh, something wrong")
                    
    print (count)
    print ("end")



In [2]:
day_one(1)   # 466
day_one(150)   # 750

start
466.0
end
start
repeat found for 750.0


### Day 2

In [3]:
def day_two():
    with open("input (1).txt","r") as filein:
        two_count = 0
        three_count = 0
        for line in filein:
            letter_list = (list(line)[:-1])
            for letter in letter_list: 
                if letter_list.count(letter) == 2:
                    two_count += 1
                    break
                else:
                    continue
            for letter in letter_list:                        
                if letter_list.count(letter) == 3:
                    three_count += 1
                    break
                else:
                    continue
                
    checksum = two_count * three_count
    print (checksum)
    
    ID_list = []
    with open("input (1).txt","r") as filein:
        for line in filein:
            ID_list.append(line[:-1])
            
    for ID1_index,ID1 in enumerate(ID_list):
        for ID2 in ID_list:
            # this is the juice. Go through removing one letter at a time, from both arrays, and see if they match.
            for letter_index in range(len(list((ID1)))):
                if (list(ID1)[:letter_index]+list(ID1)[letter_index+1:]) == (list(ID2)[:letter_index]+list(ID2)[letter_index+1:]):
                    if ID1 != ID2:
                        print ("match found:"+ID1+"/"+ID2)
                        return

                else:
                    continue
                
            
    

In [4]:
day_two() # 5166
          # cypueihajytordkgzxfqplbwn

5166
match found:cypueihajytotrdkgzxfqplbwn/cypueihajytomrdkgzxfqplbwn
match found:cypueihajytomrdkgzxfqplbwn/cypueihajytotrdkgzxfqplbwn


### Day 3

In [5]:
import numpy as np
import re

def day_three():
    fabric = np.zeros([1000,1000])
    with open("input (2).txt","r") as filein:
        for line in filein:
            ID = line.split()[0]
            left=int(re.search(r'(\d*)\,(\d*)',line.split()[2]).group(1))
            top=int(re.search(r'(\d*)\,(\d*)',line.split()[2]).group(2))
            width=int(re.search(r'(\d*)x(\d*)',line.split()[3]).group(1))
            height=int(re.search(r'(\d*)x(\d*)',line.split()[3]).group(2))
            for i in range(height):
                for j in range(width):
                    fabric[top+i][left+j]+=1
    # np.where is suuupppeerr useful here             
    overlap=fabric[np.where(fabric>1)]
    print (len(overlap))
    
## part two
    a=np.where(fabric==1)
    fabric_indices=[]
    # list of array indices where equal to 1
    for i,j in zip(a[0],a[1]):
        fabric_indices.append([i,j])
    
    with open("input (2).txt","r") as filein:                
        for line in filein:
            fabric_individual = np.zeros([1000,1000])
            ID = line.split()[0]
            left=int(re.search(r'(\d*)\,(\d*)',line.split()[2]).group(1))
            top=int(re.search(r'(\d*)\,(\d*)',line.split()[2]).group(2))
            width=int(re.search(r'(\d*)x(\d*)',line.split()[3]).group(1))
            height=int(re.search(r'(\d*)x(\d*)',line.split()[3]).group(2))
            for i in range(height):
                for j in range(width):
                    fabric_individual[top+i][left+j]=1
                    
            a=np.where(fabric_individual==1)
            individual_indices=[]
            # list of array indices for particular fabric cut where equal to 1
            for i,j in zip(a[0],a[1]):
                individual_indices.append([i,j])
            
            # if all of the array indices are also 1 in the fabric_array, then we have our match
            if all(elem in fabric_indices for elem in individual_indices):
                print (ID)
                return
    
                
    
# for sanity checks..
#    with open("fabric.txt","w") as fileout:
#        with np.printoptions(threshold=np.inf):
#            fileout.write(str(overlap))

In [ ]:
day_three() #118322
            # 1178

### Day 4

In [7]:
import pandas as pd

# I've probably went a bit overkill here, but I went from clarity over brevity.

def day_four():

    # read in data and sort by datetime stamp       
    df=pd.read_csv("input (3).txt",delimiter="]",header=None,names=["time","event"])
    df=df.sort_values(by='time')
    
    # create ID column row
    df['ID'] = pd.Series(np.random.randint(0,10,size=len(df['time'])))
                         
    # set ID column row
    for index, row in df.iterrows():
        if 'Guard' in row['event']:
            ID_number = int(row['event'].split()[1][1:])
            df.set_value(index, 'ID', ID_number)
        else:
            df.set_value(index, 'ID', ID_number)
    
    # iterate through guard ID_numbers
    guard_total=[]
    guard_break_down=[]
    for ID_number in df.ID.unique():
        # re-set index for each individual guard
        df_guard = df[df['ID'] == ID_number].reset_index(drop=True)   
        # set a tally for each minute between midnight and 1am
        sleep_count = np.zeros(60)      
        for index, row in df_guard.iterrows():
            if 'asleep' in row['event']:
                # check it's not the final row in the dataframe
                if (index+1 != len(df_guard)):
                    # add 1 to the minute tally between asleep and waking minutes
                    if 'wakes' in df_guard.iloc[index+1]['event']:
                        start=int(df_guard.iloc[index]['time'].split()[1][-2:])
                        end=int(df_guard.iloc[index+1]['time'].split()[1][-2:])
                        sleep_count[start:end]+=1
                    # if final event for the day, add 1 to the minute tally from minute asleep to 1am (though don't think this happens)
                    else:
                        start=int(df_guard.iloc[index]['time'].split()[1][-2:])
                        sleep_count[start:]+=1
                # if final row in dataframe, add 1 to the minute tally from minute asleep to 1am (though don't think this happens)
                else:
                    start=int(df_guard.iloc[index]['time'].split()[1][-2:])
                    sleep_count[start:]+=1

        guard_total.append([ID_number,sleep_count.sum()])
        guard_break_down.append([ID_number,sleep_count])
    return np.array(guard_total), guard_break_down

In [8]:
# I got sloppy here and done stuff by inspection, but this was the easiest way (for me)

guard_total, guard_break_down = day_four()
print (guard_total)
print (max(guard_total[:,1]))
# from inspection
# ID = 131
# minute = 36
# part 1: 4716

guard_max=[]
for break_down in guard_break_down:
    print (break_down[0],max(break_down[1]))
    
# from inspection
# ID = 2389
# minute = 49
# part 2: 117061

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:18: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:20: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead


[[1487.  461.]
 [1493.  429.]
 [ 103.  291.]
 [2389.  427.]
 [3391.  366.]
 [3253.  229.]
 [ 641.  402.]
 [ 131.  489.]
 [2797.  349.]
 [2437.  358.]
 [2447.  448.]
 [ 619.  375.]
 [1613.  334.]
 [1033.  252.]
 [2969.  235.]
 [ 443.  371.]
 [2089.  330.]
 [1373.  367.]
 [2417.  432.]
 [3023.  150.]
 [2683.    0.]
 [1973.    0.]
 [1913.    0.]]
489.0
1487 13.0
1493 14.0
103 10.0
2389 15.0
3391 11.0
3253 9.0
641 14.0
131 14.0
2797 9.0
2437 12.0
2447 11.0
619 12.0
1613 11.0
1033 10.0
2969 7.0
443 13.0
2089 9.0
1373 11.0
2417 11.0
3023 5.0
2683 0.0
1973 0.0
1913 0.0


### Day 5

In [16]:
## at first I tried iterating through each element of the string and comparing letter to letters before and after.
## Using regex is much less clunky

import string
import re

def react_polymer(polymer_string):
    lowers=list(string.ascii_lowercase)
    proceed = True
    while proceed==True:
        proceed=False
        for letter in lowers:
            Ul = letter.upper()+letter
            if re.search(Ul,polymer_string):
                regex=re.search(r'(.*)'+Ul+'(.*)',polymer_string)
                polymer_string=regex.group(1)+regex.group(2)
                proceed=True
            lU = letter + letter.upper()
            if re.search(lU,polymer_string):
                regex=re.search(r'(.*)'+lU+'(.*)',polymer_string)
                polymer_string=regex.group(1)+regex.group(2)
                proceed=True

    return (len(polymer_string))



In [ ]:
# part one
# polymer_string = "dabAcCaCBAcCcaDA"   # example string
polymer_string = open("input (4).txt","r").read()   # question string
units=react_polymer(polymer_string) # answer 11546
print ("part one: "+str(units))

# part two
results=[]
lowers=list(string.ascii_lowercase)
for letter in lowers:
    polymer_string_reduced = re.sub(letter,'',polymer_string)
    polymer_string_reduced = re.sub(letter.upper(),'',polymer_string_reduced)
    results.append([letter,react_polymer(polymer_string_reduced)])
# I need to do this a better way than appending to lists...making an array..transforming to ints...finding min...
print ("part two: "+min([int(x) for x in np.array(results)[:,1]]))

part one: 11546
